# DATA CLEANING

`1. Importamos modulos `

In [2]:
import pandas as pd
import re
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt


`2. Leemos el documento csv`

In [3]:
df = pd.read_csv('Input/data.csv', encoding='latin-1')
df.head()

,Country,Rank,Track_id,Streams,Track Name,Artist,URL,acousticness,danceability,energy,...,valence,tempo,time_signature,duration_ms,key,mode,Artist_id,Artist_popularity,Artist_follower,Artist_img
0,global,1.0,25sgk305KZfyuqVBQIahim,1166185736,Sweet but Psycho,Ava Max,https://open.spotify.com/track/25sgk305KZfyuqV...,0.0691,0.719,0.704,...,0.628,133.002,4,187436,1,1,4npEfmQ6YuiwW1GpUmaq3F,87,1025358,https://i.scdn.co/image/4b914c6470c8458674538a...
1,global,2.0,2Fxmhks0bxGSBdJ92vM42m,1052358787,bad guy,Billie Eilish,https://open.spotify.com/track/2Fxmhks0bxGSBdJ...,0.3280,0.701,0.425,...,0.562,135.128,4,194088,7,1,6qqNVTkY8uBg9cP3Jd7DAH,98,18867904,https://i.scdn.co/image/2622edec99d68d1d141886...
2,global,3.0,6ocbgoVGwYJhOv1GgI9NsF,789094044,7 rings,Ariana Grande,https://open.spotify.com/track/6ocbgoVGwYJhOv1...,0.5920,0.778,0.317,...,0.327,140.048,4,178627,1,0,66CXWjxzNUsdJxJ2JdwvnR,97,40958727,https://i.scdn.co/image/b1dfbe843b0b9f54ab2e58...
3,global,4.0,1rgnBhdG2JDFTbYkYRZAku,764208309,Dance Monkey,Tones and I,https://open.spotify.com/track/1rgnBhdG2JDFTbY...,0.6880,0.825,0.593,...,0.540,98.078,4,209755,6,0,2NjfBq1NflQcKSeiDooVjY,92,484241,https://i.scdn.co/image/c128f5ef4d210a67610acd...
4,global,5.0,6v3KW9xbzN5yKLt9YKDYA2,763064359,SeÃ±orita,Shawn Mendes,https://open.spotify.com/track/6v3KW9xbzN5yKLt...,0.0392,0.759,0.548,...,0.749,116.967,4,190800,9,0,7n2wHs1TKAczGzO7Dd2rGr,94,23152263,https://i.scdn.co/image/6bd59cfbd3e1e6394af710...


`3. Vemos cual es la longitud del documento csv`

In [4]:
df.shape

(1717, 24)

`3. eliminamos columnas que no necesitamos `

In [18]:
data = df.drop(columns=['Country','Rank', 'Track_id','time_signature','URL','Artist_id','acousticness','danceability','energy','valence','tempo','mode','duration_ms','instrumentalness','loudness','speechiness','key','liveness', 'Artist_img'] )

In [19]:
data.columns

Index(['Streams', 'Track Name', 'Artist', 'Artist_popularity',
       'Artist_follower'],
      dtype='object')

In [20]:
data['Artist'].value_counts().head(10)

Post Malone      45
Juice WRLD       31
Ariana Grande    28
Billie Eilish    27
Taylor Swift     27
Trippie Redd     25
Ed Sheeran       25
Drake            24
Khalid           24
BeyoncÃ©         23
Name: Artist, dtype: int64

In [21]:
data.isnull().sum()

Streams              0
Track Name           0
Artist               0
Artist_popularity    0
Artist_follower      0
dtype: int64

`4. Creamos dos nuevas columnas para luego realizar el filtrado a través de los followers y los streams`

In [22]:
data['Streams1'] = pd.qcut(data['Streams'], q=4, labels=['very low', 'low', 'medium', 'high'])
data['Artist_follower'] = pd.qcut(data['Artist_follower'], q=4, labels=['very low', 'low', 'medium', 'high'])

In [30]:
def conditions(data):
    if (data['Artist'] == "Ava Max") or (data['Artist'] == "Billie Eilish") or (data['Artist'] == "Ariana Grande") or (data['Artist'] == "Ed Sheeram") or (data['Artist'] == "Post Malone") or (data['Artist'] == "Lil Nas X") or (data['Artist'] == "Khalid") or (data['Artist'] == "Queen") or (data['Artist'] == "XXXTENTACION") or (data['Artist'] == "Shawn Mendes"):
        return 1
    else:
        return 0

In [31]:
data['Top10'] = data.apply(conditions, axis=1)

In [32]:
data

,Streams,Track Name,Artist,Artist_popularity,Artist_follower,Streams1,Top10
0,1166185736,Sweet but Psycho,Ava Max,87,very low,high,1
1,1052358787,bad guy,Billie Eilish,98,high,high,1
2,789094044,7 rings,Ariana Grande,97,high,high,1
3,764208309,Dance Monkey,Tones and I,92,very low,high,0
4,763064359,SeÃ±orita,Shawn Mendes,94,high,high,1
...,...,...,...,...,...,...,...
1712,554971,Cheap Thrills,Sia,90,high,very low,0
1713,544655,HaifischNikez Allstars,LX,71,very low,very low,0
1714,536886,"Stayin' Alive - From ""Saturday Night Fever"" So...",Bee Gees,79,low,very low,0
1715,536447,Medley da Gaiola - Dennis DJ Remix,MC Kevin o Chris,84,very low,very low,0


In [33]:
data.to_csv("Input/spotify_clean.csv")

`5. Agrupamos los artistas que más canciones han tenido y sumamos sus streams`

In [11]:
artists_streams = pd.DataFrame(data.groupby(["Artist"]).agg({'Track Name':'count','Streams':'sum'}))

In [12]:
artists_streams_save= artists_streams.sort_values('Streams', ascending=False)
artists_streams_save.head(10)

,Track Name,Streams
Artist,,
Billie Eilish,27,4497652401
Post Malone,45,4244240516
Ariana Grande,28,3339800155
Ed Sheeran,25,3118912731
Lil Nas X,20,1777338768
Shawn Mendes,8,1729841617
Ava Max,2,1573833722
Khalid,24,1435399324
XXXTENTACION,18,1370067912


`6. Quitamos el espacio de Track Name para que luego no nos de problemas`

In [13]:
artists_streams_save = artists_streams_save.rename(columns={'Track Name': 'num_songs'})

In [18]:
artists_streams_save.head(10)

,num_songs,Streams
Artist,,
Billie Eilish,27,4497652401
Post Malone,45,4244240516
Ariana Grande,28,3339800155
Ed Sheeran,25,3118912731
Lil Nas X,20,1777338768
Shawn Mendes,8,1729841617
Ava Max,2,1573833722
Khalid,24,1435399324
XXXTENTACION,18,1370067912


`7. Nos quedamos unicamente con los 10 artistas con más reproduciones y guardamos el documento`

In [15]:
artists = artists_streams_save.nlargest(10, 'Streams')

In [16]:
artists.to_csv("src/artists_streams.csv")